<a href="https://colab.research.google.com/github/jessilver/Organizacao_Computadores/blob/develop/Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Faça uma cópia no seu drive

Importante: as instalações precisam ser feita toda vez que for iniciada uma nova seção no Colab

# Jessé Eliseu Nunes da Silva

# Como usar:

As implementações foram separadas e partes, para que a interface reconheça a parte que desesja utilizar bastas "ativa-la", trocando o valor da variavel de status de false para true

# Parte 1


In [39]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import struct  # Necessário para as operações de ponto flutuante

# ===================================================================
# 1. DEFINIÇÃO DOS WIDGETS DA INTERFACE
# ===================================================================
code_area = widgets.Textarea(
    value="",
    placeholder="Escreva seu código assembly aqui...",
    description="Code:",
    layout=widgets.Layout(width="100%", height="200px")
)

run_button = widgets.Button(description="Run Program", button_style="success")
reg_button = widgets.Button(description="Show Registers", button_style="info")
mem_button = widgets.Button(description="Show Memory", button_style="info")

start_addr = widgets.IntText(value=0, description="Start Addr")
mem_range = widgets.IntText(value=10, description="Range")

output_area = widgets.Output()


# ===================================================================
# 2. FUNÇÕES AUXILIARES E DE EXIBIÇÃO
# ===================================================================

# -- Funções Auxiliares para Ponto Flutuante --
def float_to_int_bits(f):
    """Converte um float Python para sua representação em bits (inteiro de 32 bits)."""
    return struct.unpack('I', struct.pack('f', float(f)))[0]

def int_bits_to_float(i):
    """Converte uma representação em bits (inteiro de 32 bits) para um float Python."""
    try:
        # Garante que estamos trabalhando com 32 bits para o unpack
        return struct.unpack('f', struct.pack('I', i & 0xFFFFFFFF))[0]
    except (struct.error, ValueError):
        # Retorna NaN se os bits não formarem um float válido
        return float('nan')

# -- Convert 32-bit integer to signed --
def to_signed32(value):
    """Converte um valor inteiro para sua representação com sinal de 32 bits."""
    # Trunca para 32 bits
    value = value & 0xFFFFFFFF
    if value & (1 << 31):
        return value - (1 << 32)
    return value

# -- Funções de Exibição --
def show_registers(regs):
    """
    Exibe os 32 registradores em 4 colunas, mostrando tanto o valor
    inteiro quanto a sua interpretação em ponto flutuante (FP).
    """
    output_widgets = []

    for i in range(0, 32, 8):
        start_reg = i
        end_reg = i + 8
        register_values = [regs.read(j) for j in range(start_reg, end_reg)]
        data = {
            "Reg": [f"x{j}" for j in range(start_reg, end_reg)],
            "Valor Int": [to_signed32(val) for val in register_values],
            "Valor FP": [f"{int_bits_to_float(val):.4f}" for val in register_values]
        }
        df = pd.DataFrame(data).set_index("Reg")
        out = widgets.Output()
        with out:
            display(df)
        output_widgets.append(out)

    register_display = widgets.HBox(output_widgets)

    with output_area:
        clear_output(wait=True)
        display(register_display)

def show_memory(mem, start=0, rng=10):
    """Exibe um trecho da memória."""
    try:
        data = {"Endereço": list(range(start, start + rng)),
                "Valor": [mem.load_word(i) for i in range(start, start + rng)]}
        df = pd.DataFrame(data)
        with output_area:
            clear_output(wait=True)
            display(df)
    except IndexError as e:
        with output_area:
            clear_output(wait=True)
            print(f"Erro ao acessar memória: {e}")


# ===================================================================
# 3. CLASSES DO SIMULADOR (MEMÓRIA E REGISTRADORES)
# ===================================================================

class SimpleMemory:
    def __init__(self, size_in_words):
        self.size = size_in_words
        self.mem = [0] * size_in_words

    def load_word(self, address):
        if 0 <= address < self.size:
            return self.mem[address]
        else:
            raise IndexError(f"Endereço de memória inválido: {address}")

    def store_word(self, address, value):
        if 0 <= address < self.size:
            # Garante que o valor armazenado está no formato 32-bit
            self.mem[address] = value & 0xFFFFFFFF
        else:
            raise IndexError(f"Endereço de memória inválido: {address}")

class RegisterFile:
    def __init__(self):
        self.regs = [0] * 32

    def read(self, reg_num):
        if 0 <= reg_num < 32:
            return self.regs[reg_num]
        else:
            raise IndexError(f"Número de registrador inválido: x{reg_num}")

    def write(self, reg_num, value):
        if reg_num == 0: # Registrador x0 é sempre 0
            return
        if 0 < reg_num < 32:
            # Garante que o valor armazenado está no formato 32-bit
            self.regs[reg_num] = value & 0xFFFFFFFF
        else:
            raise IndexError(f"Número de registrador inválido: x{reg_num}")


# ===================================================================
# 4. LÓGICA PRINCIPAL DO SIMULADOR (PARSE E EXECUTE)
# ===================================================================

def execute_type_1_instruction(instr, regs, mem):
    op = instr["op"]
    info = ""

    INT32_MIN, INT32_MAX = -2**31, 2**31 - 1

    # --- OPERAÇÕES DE INTEIROS ---
    if op in ["ADD", "ADDI", "SUB", "MUL", "DIV"]:
        rs1_val = to_signed32(regs.read(instr["rs1"]))

        if op == "ADDI":
            rs2_val = instr["imm"]
            op_symbol = '+'
        else:
            rs2_val = to_signed32(regs.read(instr["rs2"]))
            op_map = {"ADD": '+', "SUB": '-', "MUL": '*', "DIV": '/'}
            op_symbol = op_map[op]

        if op in ["ADD", "ADDI"]:
            result = rs1_val + rs2_val
            if not (INT32_MIN <= result <= INT32_MAX):
                raise OverflowError(f"Overflow na soma: {rs1_val} + {rs2_val}")

        elif op == "SUB":
            result = rs1_val - rs2_val
            if not (INT32_MIN <= result <= INT32_MAX):
                raise OverflowError(f"Overflow na subtração: {rs1_val} - {rs2_val}")

        elif op == "MUL":
            result = rs1_val * rs2_val
            if not (INT32_MIN <= result <= INT32_MAX):
                raise OverflowError(f"Overflow na multiplicação: {rs1_val} * {rs2_val}")

        elif op == "DIV":
            if rs2_val == 0:
                raise ValueError("Divisão por zero")
            if rs1_val == INT32_MIN and rs2_val == -1:
                raise OverflowError(f"Overflow na divisão: {rs1_val} / {rs2_val}")
            result = int(rs1_val / rs2_val)

        regs.write(instr["rd"], result)
        info = f"x{instr['rd']} = {rs1_val} {op_symbol} {rs2_val} -> {result}"

    # --- OPERAÇÕES DE PONTO FLUTUANTE ---
    elif op in ["ADD.S", "MUL.S"]:
        bits1 = regs.read(instr["rs1"])
        bits2 = regs.read(instr["rs2"])

        float1 = int_bits_to_float(bits1)
        float2 = int_bits_to_float(bits2)

        if op == "ADD.S":
            result_float = float1 + float2
            op_symbol = '+'
        elif op == "MUL.S":
            result_float = float1 * float2
            op_symbol = '*'

        result_bits = float_to_int_bits(result_float)

        regs.write(instr["rd"], result_bits)
        info = f"x{instr['rd']} = {float1:.2f} {op_symbol} {float2:.2f} -> {result_float:.2f} (bits: {result_bits})"

    # --- OPERAÇÕES DE MEMÓRIA ---
    elif op == "LW":
        addr = instr["addr"]
        value = mem.load_word(addr)
        regs.write(instr["rd"], value)
        info = f"x{instr['rd']} = mem[{addr}] (Valor: {value})"

    elif op == "SW":
        addr = instr["addr"]
        value = regs.read(instr["rs1"])
        mem.store_word(addr, value)
        info = f"mem[{addr}] = x{instr['rs1']} (Valor: {value})"

    else:
        info = f"Instrução desconhecida {op}"

    return info

def parse_program(asm_code):
    program = []
    for line in asm_code.strip().splitlines():
        line = line.strip().split('#')[0] # Remove comentários
        if not line:
            continue

        parts = line.replace(",", "").split()
        op = parts[0].upper()

        if op in ["ADD", "SUB", "MUL", "DIV", "ADD.S", "MUL.S"]:
            rd = int(parts[1][1:])
            rs1 = int(parts[2][1:])
            rs2 = int(parts[3][1:])
            instr = {"op": op, "rd": rd, "rs1": rs1, "rs2": rs2}

        elif op == "ADDI":
            rd = int(parts[1][1:])
            rs1 = int(parts[2][1:])
            imm = int(parts[3])
            instr = {"op": op, "rd": rd, "rs1": rs1, "imm": imm}

        elif op == "LW":
            rd = int(parts[1][1:])
            addr = int(parts[2])
            instr = {"op": op, "rd": rd, "addr": addr}

        elif op == "SW":
            rs1 = int(parts[1][1:])
            addr = int(parts[2])
            instr = {"op": op, "rs1": rs1, "addr": addr}

        else:
            raise ValueError(f"Instrução desconhecida: {line}")

        program.append(instr)
    return program

def run_program(asm_code, regs, mem):
    try:
        program = parse_program(asm_code)
        cycle = 0
        for instr in program:
            cycle += 1
            info = execute_type_1_instruction(instr, regs, mem)
            print(f"Ciclo {cycle}: {instr} -> {info}")
        return regs, mem
    except (ValueError, IndexError, OverflowError) as e:
        print(f"ERRO NA EXECUÇÃO: {e}")
        return regs, mem


# ===================================================================
# 5. HANDLERS E CONFIGURAÇÃO DA INTERFACE
# ===================================================================

# -- Handlers dos botões --
def run_program_handler(b):
    with output_area:
        clear_output(wait=True)
        print("Inicializando registradores e memória...")

    global regs, mem
    regs = RegisterFile()
    mem = SimpleMemory(64)

    code = code_area.value

    # A mensagem de execução é impressa na saída principal, não no output_area
    print("--- Log de Execução ---")
    run_program(code, regs, mem)
    print("--- Fim da Execução ---")

    with output_area:
        print("Programa executado. Use os botões para inspecionar o estado.")

def reg_button_handler(b):
    show_registers(regs)

def mem_button_handler(b):
    show_memory(mem, start=start_addr.value, rng=mem_range.value)

# -- Liga botões aos handlers --
run_button.on_click(run_program_handler)
reg_button.on_click(reg_button_handler)
mem_button.on_click(mem_button_handler)

# Inicialização global para evitar erros no primeiro clique
regs = RegisterFile()
mem = SimpleMemory(64)

# -- Layout da Interface --
ui = widgets.VBox([
    code_area,
    widgets.HBox([run_button, reg_button, mem_button, start_addr, mem_range]),
    output_area
])

# -- Exibe a interface --
display(ui)

--- Log de Execução ---
Ciclo 1: {'op': 'ADDI', 'rd': 1, 'rs1': 0, 'imm': 100} -> x1 = 0 + 100 -> 100
Ciclo 2: {'op': 'ADDI', 'rd': 2, 'rs1': 0, 'imm': 10} -> x2 = 0 + 10 -> 10
Ciclo 3: {'op': 'DIV', 'rd': 3, 'rs1': 1, 'rs2': 2} -> x3 = 100 / 10 -> 10
Ciclo 4: {'op': 'ADDI', 'rd': 10, 'rs1': 0, 'imm': 1075838976} -> x10 = 0 + 1075838976 -> 1075838976
Ciclo 5: {'op': 'ADDI', 'rd': 11, 'rs1': 0, 'imm': 1080033280} -> x11 = 0 + 1080033280 -> 1080033280
Ciclo 6: {'op': 'ADD.S', 'rd': 12, 'rs1': 10, 'rs2': 11} -> x12 = 2.50 + 3.50 -> 6.00 (bits: 1086324736)
Ciclo 7: {'op': 'MUL.S', 'rd': 13, 'rs1': 10, 'rs2': 11} -> x13 = 2.50 * 3.50 -> 8.75 (bits: 1091305472)
--- Fim da Execução ---


# Parte 2

# Parte 3

# Parte 4

# Parte 5